In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session() 

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 2s 29us/sample - loss: 0.3143


In [5]:
predictions = model.predict(x_test)

In [12]:
predictions[:1]

array([[2.0078852e-04, 3.8760394e-08, 2.0996991e-03, 2.3580731e-03,
        1.7406029e-07, 4.1207833e-05, 4.5558304e-09, 9.9482697e-01,
        2.5986974e-05, 4.4698527e-04]], dtype=float32)

In [6]:
model.save('sample_model.h5')

In [7]:
# Recreate the exact same model purely from the file
new_model = keras.models.load_model('sample_model.h5')

In [8]:
import numpy as np

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

In [13]:
new_predictions[:1]

array([[2.0078852e-04, 3.8760394e-08, 2.0996991e-03, 2.3580731e-03,
        1.7406029e-07, 4.1207833e-05, 4.5558304e-09, 9.9482697e-01,
        2.5986974e-05, 4.4698527e-04]], dtype=float32)

### Export to Tensorflow SavedModel format

In [11]:
# Export the model to a SavedModel
model.save('sample_model_tf_format', save_format='tf')

# Recreate the exact same model
new_model = keras.models.load_model('sample_model_tf_format')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer state is preserved as well:
# you can resume training where you left off.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: sample_model_tf_format/assets


In [13]:
# save and reload architecture
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# Note that the model state is not preserved! We only saved the architecture.
new_predictions = reinitialized_model.predict(x_test)
abs(np.sum(predictions - new_predictions)) > 0.

True

In [14]:
reinitialized_model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

In [16]:
json_config

'{"class_name": "Model", "config": {"name": "3_layer_mlp", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "name": "digits"}, "name": "digits", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense_1", "inbound_nodes": [[["digits", 0, 0, {}]]]}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "co

In [17]:
weights = model.get_weights()  # Retrieves the state of the model.
model.set_weights(weights)  # Sets the state of the model.

In [18]:
weights

[array([[ 0.04263405, -0.03755778,  0.03282901, ..., -0.0515887 ,
         -0.02675153,  0.05067324],
        [ 0.07283438,  0.03780606, -0.03740942, ..., -0.06465846,
          0.01964456,  0.06260148],
        [-0.02692432,  0.06012038, -0.02780378, ..., -0.06492595,
          0.03084081, -0.01388476],
        ...,
        [-0.01205482,  0.07091527,  0.08052897, ...,  0.05860315,
         -0.04708352,  0.03180381],
        [ 0.08074566,  0.02307171, -0.07959799, ...,  0.05301119,
          0.00794027, -0.08015244],
        [ 0.031083  ,  0.05887123, -0.04255563, ...,  0.02138579,
         -0.04305201,  0.02202586]], dtype=float32),
 array([-0.00538773, -0.03620107,  0.02981858, -0.04313725,  0.02045698,
         0.02561693, -0.01211659,  0.00895781,  0.00880084,  0.04444873,
        -0.029094  ,  0.05318054,  0.04100849,  0.06241094,  0.07164851,
         0.03763188,  0.00430492, -0.03925337,  0.08091519,  0.05459992,
         0.00496048,  0.04378297,  0.027511  ,  0.06991988,  0.043

In [28]:
# recreating same model from config and weights
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
print(np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6))
#abs(np.sum(predictions - new_predictions)) > 0.
# Note that the optimizer was not preserved,
# so the model should be compiled anew before training
# (and the optimizer will start from a blank state).

None


In [29]:
new_predictions

array([[9.79534070e-06, 1.25883020e-07, 1.56313693e-03, ...,
        9.97494936e-01, 9.85438783e-06, 9.01855310e-05],
       [1.25972965e-05, 4.81596449e-04, 9.98120964e-01, ...,
        2.98465863e-10, 4.69943625e-05, 9.57226298e-10],
       [1.34523971e-05, 9.95399773e-01, 1.56472204e-03, ...,
        6.48768968e-04, 6.52616087e-04, 1.03887010e-04],
       ...,
       [4.25802796e-07, 1.86264444e-07, 3.60167888e-07, ...,
        8.44776587e-05, 4.68208513e-04, 8.03571660e-03],
       [7.13831378e-06, 7.19828085e-06, 2.27954570e-06, ...,
        3.51587524e-08, 5.03550749e-04, 1.72734588e-07],
       [5.44922258e-08, 1.34508289e-08, 8.97230530e-06, ...,
        2.47154491e-10, 1.49269475e-09, 2.57688826e-09]], dtype=float32)

In [23]:
np.sum(new_predictions)

10000.0

In [32]:
# save json to the disk
# Save JSON config to disk
json_config = model.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('path_to_my_weights.h5')

# Reload the model from the 2 files we saved
with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer was not preserved.

In [33]:
json_config

'{"class_name": "Model", "config": {"name": "3_layer_mlp", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "name": "digits"}, "name": "digits", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense_1", "inbound_nodes": [[["digits", 0, 0, {}]]]}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "co